In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import keras; print(keras.__version__)
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras import models
from keras import layers
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

C:\Users\Johnny\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


2.2.4


In [3]:
#import data

epa = pd.read_csv("C:/Users/Johnny/Documents/UNH Notes and Documents 2018/Spring 2018/Neural Networks and ML/quiz/epa_cleaned_Data_2.csv", sep=',')   

In [4]:
#EDA

from collections import Counter
print(Counter(epa["Three_cat"]))
print(epa["Three_cat"].value_counts(normalize=True) * 100)

Counter({'Low': 13861, 'Medium': 5377, 'High': 370, nan: 130})
Low       70.690534
Medium    27.422481
High       1.886985
Name: Three_cat, dtype: float64


In [5]:
#subset relevant columns 
epa_numerical = epa[["Test_Veh_Displacement__L_", "Rated_Horsepower", "__of_Gears", "Equivalent_Test_Weight__lbs__", "Axle_Ratio", "N_V_Ratio", "Three_cat", "THC_bin", "CO_bin", "CO2_cat", "NOx_cat"]]

In [6]:
#subset l,m,h

vals = ['Low', 'Medium', 'High']

lmh = epa_numerical["Three_cat"].isin(vals)
epa_numerical = epa_numerical[lmh]

In [7]:
print(epa_numerical["Three_cat"].value_counts(normalize=True) * 100)
print(Counter(epa["Three_cat"]))
print(epa_numerical.shape)

epa_numerical = epa_numerical.dropna()

Low       70.690534
Medium    27.422481
High       1.886985
Name: Three_cat, dtype: float64
Counter({'Low': 13861, 'Medium': 5377, 'High': 370, nan: 130})
(19608, 11)


In [8]:
#one hot encoding

epa_numerical = pd.get_dummies(epa_numerical, columns=["THC_bin", "CO_bin", "CO2_cat", "NOx_cat"], drop_first=False)

In [9]:
#subset data into two class dataframes
High_Low = (epa_numerical.loc[epa_numerical['Three_cat'].isin(['Low','High'])])

Low = (epa_numerical.loc[epa_numerical['Three_cat'].isin(['Low'])])
LowSample = Low.sample(frac=0.1)

Medium = (epa_numerical.loc[epa_numerical['Three_cat'].isin(['Medium'])])
MedSample = Medium.sample(frac=0.1)

lowmed = [Medium, LowSample]
medlow = [MedSample, Low]

Low_Medium = pd.concat(lowmed)
Medium_Low = pd.concat(medlow)

In [10]:
print(Counter(Low_Medium["Three_cat"]))
print(Counter(High_Low["Three_cat"]))
print(Counter(Medium_Low["Three_cat"]))

Counter({'Medium': 5160, 'Low': 1261})
Counter({'Low': 12611, 'High': 322})
Counter({'Low': 12611, 'Medium': 516})


In [11]:
# ADASYN function

from imblearn.over_sampling import SMOTE, ADASYN

def ten_adasyn(dataframe):
    dataframe = dataframe.sample(frac=0.7)
    X = dataframe.loc[:, dataframe.columns != "Three_cat"]
    Y = dataframe["Three_cat"]
    X_adasyn, y_adasyn = ADASYN().fit_sample(X,Y)
    data_adasyn = pd.concat([pd.DataFrame(X_adasyn, columns = ['Test_Veh_Displacement__L_', 'Rated_Horsepower', '__of_Gears','Equivalent_Test_Weight__lbs__', 'Axle_Ratio', 'N_V_Ratio','THC_bin_High', 'THC_bin_Low', 'CO_bin_High', 'CO_bin_Low','CO2_cat_High', 'CO2_cat_Low', 'CO2_cat_Medium', 'NOx_cat_High','NOx_cat_Low', 'NOx_cat_Medium']), pd.DataFrame(y_adasyn, columns = ['Three_cat'])],axis = 1)
    dataframe = data_adasyn
    return dataframe

In [12]:
#run adasyn 5 times
High_Low1 = ten_adasyn(High_Low)
High_Low2 = ten_adasyn(High_Low)
High_Low3 = ten_adasyn(High_Low)
High_Low4 = ten_adasyn(High_Low)
High_Low5 = ten_adasyn(High_Low)

#concatenate datasets
hl = [High_Low1, High_Low2, High_Low3, High_Low4, High_Low5]
Highlow = pd.concat(hl)

In [13]:
#run adasyn 5 times

Lowmedium1 = ten_adasyn(Low_Medium)
Lowmedium2 = ten_adasyn(Low_Medium)
Lowmedium3 = ten_adasyn(Low_Medium)
Lowmedium4 = ten_adasyn(Low_Medium)
Lowmedium5 = ten_adasyn(Low_Medium)

#concatenate datasets
lm = [Lowmedium1, Lowmedium2, Lowmedium3, Lowmedium4, Lowmedium5]
LowMedium = pd.concat(lm)

In [14]:
#run adasyn 5 times

Medium_Low1 = ten_adasyn(Medium_Low)
Medium_Low2 = ten_adasyn(Medium_Low)
Medium_Low3 = ten_adasyn(Medium_Low)
Medium_Low4 = ten_adasyn(Medium_Low)
Medium_Low5 = ten_adasyn(Medium_Low)

#concatenate datasets
ml = [Medium_Low1, Medium_Low2, Medium_Low3, Medium_Low4, Medium_Low5]
MediumLow = pd.concat(ml)

In [15]:
#concatenate all datafrmaes and drop duplicates

all_frames = [Highlow,LowMedium,MediumLow]
oversampled_df = pd.concat(all_frames)
oversampled_df = oversampled_df.drop_duplicates()
oversampled_df.shape

(95855, 17)

In [255]:
#round one hot encoded variables
oversampled_df = oversampled_df.round({'THC_bin_High':0, 'THC_bin_Low':0, 'CO_bin_High':0, 'CO_bin_Low':0,
       'CO2_cat_High':0, 'CO2_cat_Low':0, 'CO2_cat_Medium':0, 'NOx_cat_High':0,
       'NOx_cat_Low':0, 'NOx_cat_Medium':0})

In [264]:
##THC Model##
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score


#subset and split
X = oversampled_df[['Test_Veh_Displacement__L_', 'Rated_Horsepower', '__of_Gears',
       'Equivalent_Test_Weight__lbs__', 'Axle_Ratio', 'N_V_Ratio']]

y = oversampled_df[['THC_bin_High']]


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state = 0)

#scale data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#contruct model
thc_model = models.Sequential()
thc_model.add(Dense(6, input_dim=6, activation='relu')) 
thc_model.add(Dense(30, activation='relu'))
thc_model.add(Dense(1, activation='sigmoid'))

thc_model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])

# fit model
thc_model.fit(X_train, y_train, epochs=5, batch_size=128)

#validate
test_loss, test_acc = thc_model.evaluate(X_test, y_test)

print('test_acc:', test_acc)

pred = thc_model.predict(X_test)

Epoch 1/5
76747/76747 [==============================] - 1s 16us/step - loss: 0.4401 - acc: 0.8203
Epoch 2/5
76747/76747 [==============================] - 1s 9us/step - loss: 0.3794 - acc: 0.8448
Epoch 3/5
76747/76747 [==============================] - 1s 9us/step - loss: 0.3693 - acc: 0.8445
Epoch 4/5
76747/76747 [==============================] - 1s 9us/step - loss: 0.3620 - acc: 0.8447
Epoch 5/5
19187/19187 [==============================] - 0s 25us/step
test_acc: 0.8566216709292338


In [227]:
##CO Model##

X = oversampled_df[['Test_Veh_Displacement__L_', 'Rated_Horsepower', '__of_Gears',
       'Equivalent_Test_Weight__lbs__', 'Axle_Ratio', 'N_V_Ratio']]

y = oversampled_df[['CO_bin_High']]



X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

co_model = models.Sequential()
co_model.add(Dense(6, input_dim=6, activation='relu')) 
co_model.add(Dense(30, activation='relu'))
co_model.add(Dense(1, activation='sigmoid'))

co_model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])

# fit the NN
co_model.fit(X_train, y_train, epochs=7, batch_size=150)

test_loss, test_acc = co_model.evaluate(X_test, y_test)

print('test_acc:', test_acc)

Epoch 1/7
77211/77211 [==============================] - 1s 12us/step - loss: 0.4613 - acc: 0.8230
Epoch 2/7
77211/77211 [==============================] - 1s 7us/step - loss: 0.4069 - acc: 0.8334
Epoch 3/7
77211/77211 [==============================] - 1s 8us/step - loss: 0.3944 - acc: 0.8400
Epoch 4/7
77211/77211 [==============================] - 1s 10us/step - loss: 0.3874 - acc: 0.8424
Epoch 5/7
77211/77211 [==============================] - 1s 11us/step - loss: 0.3815 - acc: 0.8441
Epoch 6/7
77211/77211 [==============================] - 1s 9us/step - loss: 0.3757 - acc: 0.8455
Epoch 7/7
19303/19303 [==============================] - 0s 20us/step
test_acc: 0.8431331917318552


In [229]:
##CO2 Model##

X = oversampled_df[['Test_Veh_Displacement__L_', 'Rated_Horsepower', '__of_Gears',
       'Equivalent_Test_Weight__lbs__', 'Axle_Ratio', 'N_V_Ratio']]

y = oversampled_df[['CO2_cat_High', 'CO2_cat_Low', 'CO2_cat_Medium']]



X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

co2_model = models.Sequential()
co2_model.add(Dense(6, input_dim=6, activation='relu')) 
co2_model.add(Dense(30, activation='relu'))
co2_model.add(Dense(3, activation='softmax'))

co2_model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# fit the NN
co2_model.fit(X_train, y_train, epochs=8, batch_size=160)

test_loss, test_acc = co2_model.evaluate(X_test, y_test)

print('test_acc:', test_acc)

Epoch 1/8
77211/77211 [==============================] - 1s 17us/step - loss: 0.5942 - acc: 0.7816
Epoch 2/8
77211/77211 [==============================] - 1s 10us/step - loss: 0.5036 - acc: 0.8039
Epoch 3/8
77211/77211 [==============================] - 1s 11us/step - loss: 0.4918 - acc: 0.8046
Epoch 4/8
77211/77211 [==============================] - 1s 12us/step - loss: 0.4859 - acc: 0.8053
Epoch 5/8
77211/77211 [==============================] - 1s 11us/step - loss: 0.4819 - acc: 0.8073
Epoch 6/8
77211/77211 [==============================] - 1s 12us/step - loss: 0.4783 - acc: 0.8084
Epoch 7/8
77211/77211 [==============================] - 1s 11us/step - loss: 0.4750 - acc: 0.8091
Epoch 8/8
19303/19303 [==============================] - 0s 21us/step
test_acc: 0.8117391079137753


In [230]:
##NOX Model##

X = oversampled_df[['Test_Veh_Displacement__L_', 'Rated_Horsepower', '__of_Gears',
       'Equivalent_Test_Weight__lbs__', 'Axle_Ratio', 'N_V_Ratio']]

y = oversampled_df[['CO2_cat_High', 'CO2_cat_Low', 'CO2_cat_Medium']]



X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

nox_model = models.Sequential()
nox_model.add(Dense(6, input_dim=6, activation='relu')) 
nox_model.add(Dense(30, activation='relu'))
nox_model.add(Dense(3, activation='softmax'))

nox_model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# fit the NN
nox_model.fit(X_train, y_train, epochs=8, batch_size=160)

test_loss, test_acc = nox_model.evaluate(X_test, y_test)

print('test_acc:', test_acc)

Epoch 1/8
77211/77211 [==============================] - 1s 15us/step - loss: 0.5625 - acc: 0.7909
Epoch 2/8
77211/77211 [==============================] - 1s 11us/step - loss: 0.5066 - acc: 0.8064
Epoch 3/8
77211/77211 [==============================] - 1s 8us/step - loss: 0.4960 - acc: 0.8059
Epoch 4/8
77211/77211 [==============================] - 1s 7us/step - loss: 0.4892 - acc: 0.8053
Epoch 5/8
77211/77211 [==============================] - 1s 8us/step - loss: 0.4843 - acc: 0.8057
Epoch 6/8
77211/77211 [==============================] - 1s 8us/step - loss: 0.4805 - acc: 0.8065
Epoch 7/8
77211/77211 [==============================] - 1s 8us/step - loss: 0.4764 - acc: 0.8078
Epoch 8/8
19303/19303 [==============================] - 0s 24us/step
test_acc: 0.8151582655576649


In [1]:
# print(Counter(oversampled_df["THC_bin_High"]))
oversampled_df.tail()

NameError: name 'oversampled_df' is not defined